In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
# from LibMTL import MTLModel, MSELoss, TaskUncertaintyWeighting

In [3]:
x = nn.Parameter(torch.tensor([-0.5]*8))
x

Parameter containing:
tensor([-0.5000, -0.5000, -0.5000, -0.5000, -0.5000, -0.5000, -0.5000, -0.5000],
       requires_grad=True)

In [11]:
# Generate some random data for multi-task regression
x1 = torch.randn(100, 10)  # Input features for task 1
x2 = torch.randn(100, 10)  # Input features for task 2
y1 = torch.randn(100, 1)   # Output labels for task 1
y2 = torch.randn(100, 1)   # Output labels for task 2

# Define the neural network architecture
class MultiTaskNet(nn.Module):
    def __init__(self):
        super(MultiTaskNet, self).__init__()
        self.fc1 = nn.Linear(20, 32)
        self.fc2 = nn.Linear(32, 1)
        
    def forward(self, x1, x2):
        x = torch.cat((x1, x2), dim=1)
        # print(x.shape)
        x = nn.functional.relu(self.fc1(x))
        # print(x.shape)
        # print()
        output1 = self.fc2(x)
        output2 = self.fc2(x)
        return output1, output2

# Create the multi-task model
model = MultiTaskNet()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters())

# Train the multi-task model
for epoch in range(5):
    optimizer.zero_grad()
    output1, output2 = model(x1, x2)
    loss1 = criterion(output1, y1)
    loss2 = criterion(output2, y2)
    loss = loss1 + loss2
    loss.backward()
    optimizer.step()

# Evaluate the multi-task model
with torch.no_grad():
    output1, output2 = model(x1, x2)
    loss1 = criterion(output1, y1)
    loss2 = criterion(output2, y2)
    total_loss = loss1 + loss2
    print(total_loss)

tensor(2.0222)


In [14]:
import numpy as np
from sklearn.metrics import r2_score, mean_absolute_percentage_error

# Evaluate the multi-task model using MAPE and R-squared
with torch.no_grad():
    output1, output2 = model(x1, x2)
    y1_pred, y2_pred = output1.numpy(), output2.numpy()

    # Calculate MAPE for task 1
    mape1 = mean_absolute_percentage_error(y1, y1_pred)

    # Calculate MAPE for task 2
    mape2 = mean_absolute_percentage_error(y2, y2_pred)

    # Calculate R2 for task 1
    r2_1 = r2_score(y1, y1_pred)

    # Calculate R2 for task 2
    r2_2 = r2_score(y2, y2_pred)

    print('MAPE for task 1:', mape1)
    print('MAPE for task 2:', mape2)
    print('R-squared for task 1:', r2_1)
    print('R-squared for task 2:', r2_2)

MAPE for task 1: 1.8274181
MAPE for task 2: 1.368583
R-squared for task 1: -0.0036710662986620957
R-squared for task 2: -0.0285409777940826


In [5]:
x.shape

torch.Size([2, 6])

In [10]:
from torch_geometric.datasets import QM9
from torch_geometric.loader import DataLoader
from torch_geometric.nn import NNConv
from torch_geometric.nn.aggr import Set2Set
from torch_geometric.utils import remove_self_loops
import torch_geometric.transforms as T

In [8]:
from torch_geometric.datasets import TUDataset

dataset = TUDataset(root='.', name='QM9')
data = dataset[1]
data

Data(edge_index=[2, 6], x=[4, 0], edge_attr=[6, 0], y=[1, 19])